# Keras Overview

https://keras.io/

- 케라스는 거의 모든 종류의 딥러닝 모델을 간편하게 만들고 훈련시킬 수 있는 파이썬을 위한 딥러닝 프레임워크
    - 동일한 코드로 CPU와 GPU에서 실행할 수 있다.
    - 사용하기 쉬운 API를 가지고 있어 딥러닝 모델의 프로토타입을 빠르게 만들 수 있다.
    - 합성곱 신경망( CNN ), 순환 신경망( RNN )을 지원하며 자유롭게 조합하여 사용할 수 있다.
    - 다중 입력, 다중 출력 모델, 층의 공유, 모델 공유 등 어떤 네트워크 구조도 만들 수 있다.
- 케라스튼 MIT 라이선스를 따르므로 상업적인 프로젝트에도 자유롭게 사용할 수 있다.

- 케라스는 딥러닝 모델을 만들기 위한 고수준의 구성 요소를 제공하는 모델 수준의 라이브러리이다.
- 텐서 조작이나 미분 같은 저수준의 연산을 다루지 않는다.
- 케라스의 백엔드 엔진에서 제공하는 최적화되고 특화된 텐서 라이브러리를 사용한다.
- 케라스는 하나의 텐서 라이브러리에 국한하여 구현되어 있지 않고 모듈 구조로 구성되어 있다.   

- 여러 가지 백엔드 엔진이 케라스와 매끄럽게 연동된어 있다.
- 현재는 텐서플로( TensorFlow ), 씨아노( theano ), CNTK( Microsoft Cognitive Toolkit ) 3개를 백엔드 엔진으로 사용할 수 있다.
- 케라스로 작성된 모든 코드는 아무런 변경 없이 3개의 백엔드 중 하나를 선택해서 실행시킬 수 있다.
- 개발하는 중간에 하나의 백엔드가 특정 작업에 더 빠르다고 판단되는 언제든지 백엔드를 바꿀 수 있다.
- 널리 사용되고 확장성이 뛰어나며 상용 제품에 쓸 수 있기 때문에 대부분의 딥러닝 작업에 텐서플로로 백엔드가 기본으로 권장된다.

- TensorFlow : https://www.tensorflow.org/
- theano : http://deeplearning.net/software/theano/
- CNTK : https://github.com/Microsoft/CNTK

## 1. 전처리( Preprocessing )

#### Tokenizer() 
    - 토큰화와 정수 인코딩(단어에 대한 인덱싱)을 위해 사용된다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

t  = Tokenizer()
fit_text = "The earth is an awesome place live"
t.fit_on_texts( [ fit_text ] )

test_text = "The earth is an great place live"
sequences = t.texts_to_sequences( [ test_text ] )[ 0 ]

print("sequences : ",sequences) # great는 단어 집합(vocabulary)에 없으므로 출력되지 않는다.
print("word_index : ",t.word_index) # 단어 집합(vocabulary) 출력

#### pad_sequence() : 
    - 전체 훈련 데이터에서 각 샘플의 길이는 서로 다를 수 있다. 
    - 또는 각 문서 또는 각 문장은 단어의 수가 제각각이다. 
    - 모델의 입력으로 사용하려면 모든 샘플의 길이를 동일하게 맞추어야할 때가 있다.                                 
    - 이를 자연어 처리에서는 패딩(padding) 작업이라고 하는데, 보통 숫자 0을 넣어서 길이가 다른 샘플들의 길이를 맞춘다다. 
    - 케라스에서는 pad_sequence()를 사용한다. 
    - pad_sequence()는 정해준 길이보다 길이가 긴 샘플은 값을 일부 자르고, 정해준 길이보다 길이가 짧은 샘플은 값을 0으로 채운다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], maxlen=3, padding='pre') # 전처리가 끝나서 각 단어에 대한 정수 인코딩이 끝났다고 가정하고, 3개의 데이터를 입력으로 합니다.

- 첫번째 인자 = 패딩을 진행할 데이터
- maxlen = 모든 데이터에 대해서 정규화 할 길이
- padding = 'pre'를 선택하면 앞에 0을 채우고 'post'를 선택하면 뒤에 0을 채움.

## 워드 임베딩( Word Embedding )

- 워드 임베딩이란 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것을 말한다.

#### Embedding()
    - Embedding()은 단어를 밀집 벡터로 만드는 역할을 한다. 
    - 인공 신경망 용어로는 임베딩 층(embedding layer)을 만드는 역할을 한다. 
    - Embedding()은 정수 인코딩이 된 단어들을 입력을 받아서 임베딩을 수행한다.
    - Embedding()은 (number of samples, input_length)인 2D 정수 텐서를 입력받는다. 
    - 이 때 각 sample은 정수 인코딩이 된 결과로, 정수의 시퀀스이다. 
    - Embedding()은 워드 임베딩 작업을 수행하고 (number of samples, input_length, embedding word dimentionality)인 3D 텐서를 리턴한다.

In [ ]:
# 의사 코드(pseudo-code)로 임베딩의 개념 
#
# 문장 토큰화와 단어 토큰화
text=[['Hope', 'to', 'see', 'you', 'soon'],['Nice', 'to', 'see', 'you', 'again']]

# 각 단어에 대한 정수 인코딩
text=[[0, 1, 2, 3, 4],[5, 1, 2, 3, 6]]

# 위 데이터가 아래의 임베딩 층의 입력이 된다.
Embedding(7, 2, input_length=5)
# 7은 단어의 개수. 즉, 단어 집합(vocabulary)의 크기이다.
# 2는 임베딩한 후의 벡터의 크기이다.
# 5는 각 입력 시퀀스의 길이. 즉, input_length이다.

# 각 정수는 아래의 테이블의 인덱스로 사용되며 Embeddig()은 각 단어에 대해 임베딩 벡터를 리턴한다.
+------------+------------+
|   index    | embedding  |
+------------+------------+
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |
+------------+------------+
# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님.

- 첫번째 인자 = 단어 집합의 크기. 즉, 총 단어의 개수
- 두번째 인자 = 임베딩 벡터의 출력 차원. 결과로서 나오는 임베딩 벡터의 크기
- input_length = 입력 시퀀스의 길이

## 모델링( Modeling )

#### Sequential()
    - 인공 신경망 에서 입력층, 은닉층, 출력층을 구성한다.
    - 케라스에서는 이러한 층을 구성하기 위해 Sequential()을 사용한다. 
    - Sequential()을 model로 선언한 뒤에 model.add()라는 코드를 통해 층을 단계적으로 추가한다.

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()
model.add( 층 이름 ) # 층 추가
model.add( 층 이름 ) # 층 추가
model.add( 층 이름 ) # 층 추가

- Embedding()을 통해 생성하는 임베딩 층(embedding layer) 또한 인공 신경망의 층의 하나이므로 model.add()로 추가해야한다.

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()
model.add( Embedding( vocabulary, output_dim, input_length ) )

#### Dense() 
    - 전결합층(fully-conntected layer)을 추가한다. 
    - model.add()를 통해 추가할 수 있다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add( Dense( 1, input_dim = 3, activation = 'relu' ) )

위의 코드에서 Dense()는 한번 사용되었지만 더 많은 층을 추가할 수 있습니다. Dense()의 대표적인 인자를 보겠습니다.

- 첫번째 인자 = 출력 뉴런의 수.
- input_dim = 입력 뉴런의 수. (입력의 차원)
- activation = 활성화 함수.
    - linear : 디폴트 값으로 별도 활성화 함수 없이 입력 뉴런과 가중치의 계산 결과 그대로 출력. Ex) 선형 회귀
    - sigmoid : 시그모이드 함수. 이진 분류 문제에서 출력층에 주로 사용되는 활성화 함수.
    - softmax : 소프트맥스 함수. 셋 이상을 분류하는 다중 클래스 분류 문제에서 출력층에 주로 사용되는 활성화 함수.
    - relu : 렐루 함수. 은닉층에 주로 사용되는 활성화 함수.

위 코드에서 사용된 Dense()의 의미를 보겠습니다. 
- 첫번째 인자의 값은 1인데 이는 총 1개의 출력 뉴런을 의미한다. 
- Dense()의 두번째 인자인 input_dim은 입력층의 뉴런 수를 의미한다. 
- 이 경우에는 3이다. 3개의 입력층 뉴런과 1개의 출력층 뉴런을 만들었습니다. 이를 시각화하면 다음과 같습니다.

![Alt text]( neural_network1_final.png )

이제 Dense()를 사용하여 전결합층을 하나 더 추가해보겠습니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense( 8, input_dim = 4, activation = 'relu' ) )
model.add(Dense( 1, activation = 'sigmoid' ) ) # 출력층

이번에는 Dense()가 두번 사용되었습니다. Dense()가 처음 사용되었을 때와 추가로 사용되었을 때의 인자는 조금 다릅니다. 이제 첫번째 사용된 Dense()의 8이라는 값은 더 이상 출력층의 뉴런이 아니라 은닉층의 뉴런입니다. 뒤에 층이 하나 더 생겼기 때문입니다.

두번째 Dense()는 input_dim 인자가 없는데, 이는 이미 이전층의 뉴런의 수가 8개라는 사실을 알고있기 때문입니다. 위의 코드에서 두번째 Dense()는 마지막 층이므로, 첫번째 인자 1은 결국 출력층의 뉴런의 개수가 됩니다. 이를 시각화하면 다음과 같습니다.

![Alt text]( neural_network2_final.png )

이 외에도 LSTM, GRU, Convolution2D, BatchNormalization 등 다양한 층을 만들 수 있습니다. 

#### summary() 
    - 모델의 정보를 요약해서 보여줍니다.

In [ ]:
model.summary()

## 4. 컴파일(Compile)과 훈련(Training)

#### compile() 
    - 모델을 기계가 이해할 수 있도록 컴파일 합니다. 
    - 오차 함수와 최적화 방법, 메트릭 함수를 선택할 수 있습니다.

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

max_features = 10000

model = Sequential()
model.add( Embedding( max_features, 32 ) )
model.add( SimpleRNN( 32 ) ) # RNN
model.add( Dense( 1, activation = 'sigmoid' ) )
model.compile( optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = [ 'acc' ] )

위 코드는 임베딩층, 은닉층, 출력층을 추가하여 모델을 설계한 후에, 마지막으로 컴파일 하는 과정을 보여줍니다.

- optimizer : 훈련 과정을 설정하는 옵티마이저를 설정합니다. 'adam'이나 'sgd'와 같이 문자열로 지정할 수도 있습니다.
- loss : 훈련 과정에서 사용할 손실 함수(loss function)를 설정합니다.
= metrics : 훈련을 모니터링하기 위한 지표를 선택합니다.

대표적으로 사용되는 손실 함수와 활성화 함수의 조합은 아래와 같습니다

|문제 유형|손실 함수명|출력층의 활성화 함수명|참고 설명|
|---|---|---|---|
|회귀 문제|mean_squared_error(평균 제곱 오차)|-|-|
|다중 클래스 분류|categorical_crossentropy (범주형 교차 엔트로피)|소프트맥스|-|
|다중 클래스 분류|sparse_categorical_crossentropy|소프트맥스|범주형 교차 엔트로피와 동일하지만 이 경우 원-핫 인코딩이 된 상태일 필요없이 정수 인코딩 된 상태에서 수행 가능.|
|이진 분류|binary_crossentropy(이항 교차 엔트로피)|시그모이드|-|

#### fit()
    - 모델을 학습합니다. 
    - 모델이 오차로부터 매개 변수를 업데이트 시키는 과정을 학습, 훈련, 또는 적합(fitting)이라고 하기도 하는데, 모델이 데이터에 적합해가는 과정이기 때문입니다. 
    - 그런 의미에서 fit()은 모델의 훈련을 시작한다는 의미를 가지고 있습니다.

In [ ]:
model.fit( X_train, y_train, epochs = 10, batch_size = 32 )

- 첫번째 인자 = 훈련 데이터에 해당됩니다.
- 두번째 인자 = 지도 학습에서 레이블 데이터에 해당됩니다.
- epochs = 에포크. 에포크 1은 전체 데이터를 한 차례 훑고 지나갔음을 의미함. 정수값 기재 필요. 총 훈련 횟수를 정의합니다.
- batch_size = 배치 크기. 기본값은 32. 미니 배치 경사 하강법을 사용하고 싶지 않을 경우에는 batch_size=None을 기재합니다.

In [ ]:
model.fit( X_train, y_train, epochs = 10, batch_size = 32, verbose = 0, validation_data( X_val, y_val ) )

- validation_data(x_val, y_val) = 검증 데이터(validation data)를 사용합니다. 검증 데이터를 사용하면 각 에포크마다 검증 데이터의 정확도도 함께 출력되는데, 이 정확도는 훈련이 잘 되고 있는지를 보여줄 뿐이며 실제로 모델이 검증 데이터를 학습하지는 않습니다. 검증 데이터의 loss가 낮아지다가 높아지기 시작하면 이는 과적합(overfitting)의 신호입니다.

- validation_split= validation_data 대신 사용할 수 있습니다. 검증 데이터를 사용하는 것은 동일하지만, 별도로 존재하는 검증 데이터를 주는 것이 아니라 X_train과 y_train에서 일정 비율을 분리하여 이를 검증 데이터로 사용합니다. 역시나 훈련 자체에는 반영되지 않고 훈련 과정을 지켜보기 위한 용도로 사용됩니다. 아래는 validation_data 대신에 validation_split을 사용했을 경우를 보여줍니다.

In [ ]:
# 훈련 데이터의 20%를 검증 데이터로 사용.
model.fit( X_train, y_train, epochs = 10, batch_size = 32, verbose = 0, validation_split = 0.2 )

- verbose = 학습 중 출력되는 문구를 설정합니다.
    - 0 : 아무 것도 출력하지 않습니다.
    - 1 : 훈련의 진행도를 보여주는 진행 막대를 보여줍니다.
    - 2 : 미니 배치마다 손실 정보를 출력합니다.

## 5. 평가(Evaluation)와 예측(Prediction)

#### evaluate()
    - 테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가합니다.

In [ ]:
model.evaluate( X_test, y_test, batch_size = 32 )

- 첫번째 인자 = 테스트 데이터에 해당됩니다.
- 두번째 인자 = 지도 학습에서 레이블 테스트 데이터에 해당됩니다.
- batch_size = 배치 크기.

#### predict()
    - 임의의 입력에 대한 모델의 출력값을 확인합니다.

In [ ]:
model.predict( X_input, batch_size = 32 )

- 첫번째 인자 = 예측하고자 하는 데이터.
- batch_size = 배치 크기.

## 6. 모델의 저장(Save)과 로드(Load)

복습을 위한 스터디나 실제 어플리케이션 개발 단계에서 구현한 모델을 저장하고 불러오는 일은 중요합니다. 모델을 저장한다는 것은 학습이 끝난 신경망의 구조를 보존하고 계속해서 사용할 수 있다는 의미입니다.

#### save()
    - 인공 신경망 모델을 hdf5 파일에 저장합니다.

In [ ]:
model.save( "model_name.h5" )

#### load_model()
    - 저장해둔 모델을 불러옵니다.

In [ ]:
from tensorflow.keras.models import load_model

model = load_model( "model_name.h5" )

# Keras 함수형 API(functional API)

- 케라스를 사용하여 모델을 설계하는 방식을 sequential API를 사용하였다고 합니다. 
- sequential API는 여러층을 공유하거나 다양한 종류의 입력과 출력을 사용하는 등의 복잡한 모델을 만드는 일을 하기에는 한계가 있습니다. 
- 복잡한 모델을 생성할 수 있는 방식에는 functional API(함수형 API)를 사용합니다.

https://keras.io/getting-started/functional-api-guide/

## sequential API로 만든 모델

In [ ]:
# 소프트맥스 회귀
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add( Dense( 3, input_dim = 4, activation = 'softmax' ) )

위와 같은 방식은 직관적이고 편리하지만 단순히 층을 쌓는 것만으로는 구현할 수 없는 복잡한 인공 신경망을 구현할 수 없습니다.

## 2. functional API로 만든 모델

- functional API는 각 층을 일종의 함수(function)로서 정의합니다. 
- 각 함수를 조합하기 위한 연산자들을 제공하는데, 이를 이용하여 신경망을 설계합니다. 

### 1) 전결합 피드 포워드 신경망(Fully-connected FFNN)

- sequential API와는 다르게 functional API에서는 입력 데이터의 크기(shape)를 인자로 입력층을 정의해주어야 합니다. 
- 입력의 차원이 1인 전결합 피드 포워드 신경망(Fully-connected FFNN)을 만든다고 가정

In [ ]:
from tensorflow.keras.layers import Input

# 텐서를 리턴한다.
inputs = Input( shape = ( 10, ) )

위의 코드는 10개의 입력을 받는 입력층을 보여줍니다. 이제 위의 코드에 은닉층과 출력층을 추가

In [ ]:
from tensorflow.keras.layers import Input, Dense

inputs = Input( shape = ( 10, ) )
hidden1 = Dense( 64, activation = 'relu' )( inputs )
hidden2 = Dense( 64, activation = 'relu' )( hidden1 )
output = Dense( 1, activation = 'sigmoid' )( hidden2 )

이제 위의 코드를 하나의 모델로 구성해보겠습니다. 이는 Model에 입력 텐서와 출력 텐서를 정의하여 완성

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input( shape = ( 10, ) )
hidden1 = Dense( 64, activation ='relu' )( inputs )
hidden2 = Dense( 64, activation ='relu' )( hidden1 )
output = Dense( 1, activation ='sigmoid' )( hidden2 )
model = Model( inputs = inputs, outputs = output )

- Input() 함수에 입력의 크기를 정의합니다.
- 이전층을 다음층 함수의 입력으로 사용하고, 변수에 할당합니다.
- Model() 함수에 입력과 출력을 정의합니다.

이제 이를 model로 저장하면 sequential API를 사용할 때와 마찬가지로 model.compile, model.fit 등을 사용 가능

In [ ]:
model.compile( optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = [ 'accuracy' ] )
model.fit( data, labels )

변수명을 달리해서 FFNN을 만들어보겠습니다. 이번에는 은닉층과 출력층의 변수를 전부 x로 통일

In [ ]:
inputs = Input( shape =( 10, ) )
x = Dense( 8, activation = "relu" )( inputs )
x = Dense( 4, activation = "relu" )( x )
x = Dense( 1, activation = "linear" )( x )
model = Model( inputs, x )

### 선형 회귀(Linear Regression)

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input( shape = ( 3, ) )
output = Dense( 1, activation = 'linear' )( inputs )
linear_model = Model( inputs, output )

linear_model.compile( optimizer = 'sgd', loss = 'mse' )
linear_model.fit( x = dat_test, y = y_cts_test, epochs = 50, verbose = 0 )
linear_model.fit( x = dat_test, y = y_cts_test, epochs = 1, verbose = 1 )

### 3) 로지스틱 회귀(Logistic Regression)

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input( shape = ( 3, ) )
output = Dense( 1, activation = 'sigmoid' )( inputs )
logistic_model = Model( inputs, output )

logistic_model.compile( optimizer = 'sgd', loss = 'binary_crossentropy', metrics = [ 'accuracy' ] )
logistic_model.optimizer.lr = 0.001
logistic_model.fit( x = dat_train, y = y_classifier_train, epochs = 5, validation_data = ( dat_test, y_classifier_test ) )

### 4) 다중 입력을 받는 모델(model that accepts multiple inputs)

functional API를 사용하면 아래와 같이 다중 입력과 다중 출력을 가지는 모델도 만들 수 있습니다.

In [ ]:
# 최종 완성된 다중 입력, 다중 출력 모델의 예
model = Model( inputs = [a1, a2], outputs = [ b1, b2, b3 ] )

이번에는 다중 입력을 받는 모델을 입력층부터 출력층까지 설계해보겠습니다.

In [ ]:
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model

# 두 개의 입력층을 정의
inputA = Input( shape = ( 64, ) )
inputB = Input( shape = ( 128, ) )

# 첫번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
x = Dense( 16, activation = "relu" )( inputA )
x = Dense( 8, activation = "relu" )( x )
x = Model( inputs = inputA, outputs = x )

# 두번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
y = Dense( 64, activation = "relu" )( inputB )
y = Dense( 32, activation = "relu" )( y )
y = Dense( 8, activation = "relu" )( y )
y = Model( inputs = inputB, outputs = y )

# 두개의 인공 신경망의 출력을 연결(concatenate)
result = concatenate( [ x.output, y.output ] )

# 연결된 값을 입력으로 받는 밀집층을 추가(Dense layer)
z = Dense( 2, activation = "relu" )( result )

# 선형 회귀를 위해 activation=linear를 설정
z = Dense( 1, activation = "linear" )( z )

# 결과적으로 이 모델은 두 개의 입력층으로부터 분기되어 진행된 후 마지막에는 하나의 출력을 예측하는 모델이 됨.
model = Model( inputs = [ x.input, y.input ], outputs = z )

### 5) RNN(Recurrence Neural Network) 은닉층 사용하기

이번에는 RNN 은닉층을 가지는 모델을 설계해봅시다. 여기서는 하나의 특성(feature)에 50개의 시점(time-step)을 입력으로 받는 모델을 설계해보겠습니다.

In [ ]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

inputs = Input( shape = ( 50, 1 ) )

lstm_layer = LSTM( 10 )( inputs ) # RNN의 일종인 LSTM을 사용
x = Dense( 10, activation = 'relu' )( lstm_layer )
output = Dense( 1, activation = 'sigmoid' )( x )
model = Model( inputs = inputs, outputs = output )

### 6) 다르게 보이지만 동일한 표기

케라스의 functional API가 익숙하지 않은 상태에서 functional API를 사용한 코드를 보다가 혼동할 수 있는 점이 한 가지 있습니다. 바로 동일한 의미를 가지지만, 하나의 줄로 표현할 수 있는 코드를 두 개의 줄로 표현한 경우입니다.

In [ ]:
encoder = Dense( 128 )( input )

위 코드는 아래와 같이 두 개의 줄로 표현할 수 있습니다.

In [ ]:
encoder = Dense( 128 )
encoder( input )